In [8]:
import sys
import os

import pandas as pd
import numpy as np
import natsort
import random as rn
from tqdm import tqdm_notebook as tqdm
import tensorflow as tf
import pyeeg
from scipy import signal
from scipy.signal import welch
from scipy.integrate import simps

import matplotlib.pyplot as plt

#Keras

#Sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neural_network import MLPRegressor
from sklearn.svm import (SVC, SVR)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.utils.class_weight import compute_class_weight

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import (StratifiedKFold, KFold)

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
seed=42
np.random.seed(seed)
rn.seed(seed)
tf.set_random_seed(seed)
dir_path = os.getcwd()

# https://en.wikipedia.org/wiki/Neural_oscillation
SAMPLE_PER_SEC = 128
ALPHA_WAVE = [7.5, 12.5]
BETA_WAVE = [13, 30]
THETA_WAVE = [4, 8]
DELTA_WAVE = [1, 4]
LOW_GAMMA_WAVE = [30, 70]
HIGH_GAMMA_WAVE = [70, 150]
EPSILON =  0.0002

"""
Same for all
Min: -0.0033
Max: 0.0033


RAT 0
Class 1: 11240
Class 2: 8653
Class 3: 1707
Mean: -7.270247766565382e-06
Std: 0.00014033437213481842

RAT 1
Class 1: 11849
Class 2: 8647
Class 3: 1104
Mean: -4.883171278211801e-06
Std: 8.972252301644052e-05

RAT 2
Class 1: 11025
Class 2: 9833
Class 3: 742
Mean: -4.952989167390033e-06
Std: 0.00010911971573637134
"""



'\nSame for all\nMin: -0.0033\nMax: 0.0033\n\n\nRAT 0\nClass 1: 11240\nClass 2: 8653\nClass 3: 1707\nMean: -7.270247766565382e-06\nStd: 0.00014033437213481842\n\nRAT 1\nClass 1: 11849\nClass 2: 8647\nClass 3: 1104\nMean: -4.883171278211801e-06\nStd: 8.972252301644052e-05\n\nRAT 2\nClass 1: 11025\nClass 2: 9833\nClass 3: 742\nMean: -4.952989167390033e-06\nStd: 0.00010911971573637134\n'

In [4]:
def get_train_from_csv(csv_file):
    '''
    get a numpy array y of labels. the order follows the id of 4 second sample. 
    argument: relative path to the csv_file from the source folder.
    '''
    csv_file = os.path.join(dir_path, csv_file)
    print(f"Reading {csv_file}")
    with open(csv_file, 'r') as csvfile:
        train_reader = pd.read_csv(csvfile)
        train_reader.drop(labels="Id", axis=1, inplace=True)
        
        
    return train_reader.values

def get_target_from_csv(csv_file):
    '''
    get a numpy array y of labels. the order follows the id of 4 second sample. 
    argument: relative path to the csv_file from the source folder.
    '''
    csv_file = os.path.join(dir_path,csv_file)
    with open(csv_file, 'r') as csvfile:
        label_reader = pd.read_csv(csvfile)
        #print("Labels: ", label_reader['id'])
        y = label_reader['y']
        
    y = np.array(y)
    return y

def get_features_emg_wavelet(X):
    """
     absolute EEG power in the 1–4Hz
    (delta), 5–9Hz (theta), 10–20Hz (low beta), and 30–40Hz (high beta) bands, absolute EMG
    power in the 1–10Hz band, theta-to-delta ratio, and beta-to-delta ratio 
    """
    all_featues = []
    for i in tqdm(range(X.shape[0])):
        features = list()
        x_i = X[i,:]
        delta = bandpower(x_i, DELTA_WAVE)
        features.append(delta)
        
        theta = bandpower(x_i, THETA_WAVE)
        features.append(theta)
        
        beta = bandpower(x_i, BETA_WAVE)
        features.append(beta)
        
        features.append(bandpower(x_i, ALPHA_WAVE))
        
        features.append(theta / delta)
        features.append(beta / delta)
    
        all_featues.append(features)
    return np.array(all_featues)

def get_features_emg(X):
    all_featues = []
    for i in tqdm(range(X.shape[0])):
        features = list()
        # https://ieeexplore.ieee.org/document/7748960
        x_i = X[i,:]
        # Root Mean Square (RMS): RMS of EMG
        #features.append(mean_squared_error(x_i, np.zeros(x_i.shape)))
        
        #Integrated Absolute Value (IAV)
        #features.append(np.sum(np.abs(x_i)))
        
        # Mean Absolute Value (MAV): MAV feature can be expressed as
        #features.append(np.mean(x_i))
        
        # TBD:
        # Modified Mean Absolute Value type 1
        # Modified Mean Absolute Value type 2
        
        # Simple Square Integral (SSI): SSI is calculated as
        #features.append(np.sum(x_i ** 2))
        
        # Variance (VAR): VAR is calculated as
        #features.append(np.var(x_i))
        
        #The 3rd, 4th and 5th temporal moments
        #features.append(np.mean(x_i ** 3))
        #features.append(np.mean(x_i ** 4))
        #features.append(np.mean(x_i ** 5))
        
        # TBD
        # v-Order 
        
        # Waveform Length
        #features.append(np.sum(np.abs(np.diff(x_i))))
        
        # Average Amplitude Change
        #features.append(np.mean(np.abs(np.diff(x_i))))
        
        # Difference Absolute Standard Deviation Value
        #features.append(np.sqrt(np.mean(np.power(np.diff(x_i), 2))))
        
        # AX BASIC FEATUERS
        features.append(np.sum(np.abs(x_i) > EPSILON))
        
        
        
        all_featues.append(features)
    return np.array(all_featues)

def get_features_eeg(X):
    all_featues = []
    # NOT SURE ABOUT THIS VALUES 
    # LETS DOUBLE CHECK
    K_MAX = 6
    FREQ_BANDS = list(range(16))
    TAU = 16
    # embedding dimension
    DE = 32
    
    for i in tqdm(range(X.shape[0])):
        features = list()
        # http://pyeeg.sourceforge.net/
        x_i = X[i,:]
        
        ## !IMPORTANT! => 
        # Power Spectral Intensity (PSI) and Relative Intensity Ratio (RIR)	bin_power()	Two 1-D vectors
        
        
        # Petrosian Fractal Dimension (PFD)	pdf()	A scalar
        features.append(pyeeg.pfd(x_i))
        
        # Higuchi Fractal Dimension (HFD)	hfd()	A scalar
        features.append(pyeeg.hfd(x_i, K_MAX))
        
        # Hjorth mobility and complexity	hjorth()	Two scalars
        
        # Spectral Entropy (Shannon's entropy of RIRs)	spectral_entropy()	A scalar
        #features.append(pyeeg.spectral_entropy(x_i, FREQ_BANDS, SAMPLE_PER_SEC))
        
        # SVD Entropy	svd_entropy()	A scalar
        #features.append(pyeeg.svd_entropy(x_i, TAU, DE))
        
        # Fisher Information	fisher_info()	A scalar
        features.append(pyeeg.fisher_info(x_i, TAU, DE))
          
        # Detrended Fluctuation Analysis (DFA)	dfa()	A scalar
        features.append(pyeeg.dfa(x_i))
        
        # Hurst Exponent (Hurst)	hurst()	A scalar
        #features.append(pyeeg.hurst(x_i))
        
        # AX BASIC FEATUERS
        features.append(np.mean(x_i))
        features.append(np.std(x_i))
        features.append(np.min(x_i))
        features.append(np.max(x_i))
        features.append(np.sum(np.abs(x_i) < 0.00005))
        
        
        
        all_featues.append(features)
    return np.array(all_featues)

def split_train_validation(X, y, valid=2):
    """
    3 test subjects in train => split into 2|1
    """
    sample_count_per_subject = int(X.shape[0] / 3)
    X_train = X[:sample_count_per_subject * valid]
    y_train = y[:sample_count_per_subject * valid]
    X_valid = X[sample_count_per_subject * valid:]
    y_valid = y[sample_count_per_subject * valid:]
    
    return X_train, y_train, X_valid, y_valid


def bandpower(data, band, window_sec=4, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    band : list
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each window in seconds.
        If None, window_sec = (1 / min(band)) * 2
    relative : boolean
        If True, return the relative power (= divided by the total power of the signal).
        If False (default), return the absolute power.

    Return
    ------
    bp : float
        Absolute or relative band power.

    Examples
    ------
    1. Absolute and relative power in the delta band
        >>> delta = bandpower(data, 100, [0.5, 4])
        >>> delta_relative = bandpower(data, 100, [0.5, 4], relative=True)

    2. Delta / beta ratio
        >>> window_sec = 4
        >>> delta = bandpower(data, 100, [0.5, 4], window_sec)
        >>> beta = bandpower(data, 100, [12, 30], window_sec)
        >>> db_ratio = delta / beta
    """
    
    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if window_sec is not None:
        nperseg = window_sec * SAMPLE_PER_SEC
    else:
        nperseg = (2 / low) * SAMPLE_PER_SEC

    freqs, psd = welch(data, SAMPLE_PER_SEC, nperseg=nperseg, scaling='density')

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs < high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], freqs[idx_band])

    if relative:
        bp /= simps(psd, freqs)
    return bp

def plot_PCA_clusters(X, y_train):
    pca = PCA(n_components=2)
    pca.fit(X)
    existing_2d = pca.transform(X)
    awake_fold = existing_2d[y_train == 1]
    sleep_fold_nrem = existing_2d[y_train == 2]
    sleep_fold_rem = existing_2d[y_train == 3]

    f, (ax1, ax2, ax3) = plt.subplots(1, 3)
    ax1.plot(awake_fold[:, 0], awake_fold[:, 1], 'r.')
    ax2.plot(sleep_fold_nrem[:, 0], sleep_fold_nrem[:, 1], 'b.')
    ax3.plot(sleep_fold_rem[:, 0], sleep_fold_rem[:, 1], 'y.')
    plt.show()

    plt.plot(awake_fold[:, 0], awake_fold[:, 1], 'r.')
    plt.plot(sleep_fold_nrem[:, 0], sleep_fold_nrem[:, 1], 'b.')
    plt.plot(sleep_fold_rem[:, 0], sleep_fold_rem[:, 1], 'y.')
    plt.show()
    
def plot_fct_spec(data):
    time = np.arange(len(data)) / SAMPLE_PER_SEC
    fig, ax = plt.subplots(1, 1, figsize=(12, 4))
    plt.plot(time, data, lw=1.5, color='k')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Voltage')
    plt.xlim([time.min(), time.max()])
    plt.title('N3 sleep EEG data (F3)')

    win = 4 * SAMPLE_PER_SEC
    freqs, psd = signal.welch(data, SAMPLE_PER_SEC, nperseg=win, scaling='density')

    # Plot the power spectrum

    plt.figure(figsize=(8, 4))
    plt.plot(freqs, psd, color='k', lw=2)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power spectral density (V^2 / Hz)')
    plt.ylim([0, psd.max() * 1.1])
    plt.title("Welch's periodogram")
    plt.xlim([0, 20])
    
def plot_TSNE_clusters(X, y_train):
    N_SAMPLES = 2500
    random_index = np.random.choice(X.shape[0], N_SAMPLES)
    X = X[random_index, :]
    y_train = y[random_index]

    existing_2d = TSNE(n_components=2).fit_transform(X)
    awake_fold = existing_2d[y_train == 1]
    sleep_fold_nrem = existing_2d[y_train == 2]
    sleep_fold_rem = existing_2d[y_train == 3]

    f, (ax1, ax2, ax3) = plt.subplots(1, 3)
    ax1.plot(awake_fold[:, 0], awake_fold[:, 1], 'r.')
    ax2.plot(sleep_fold_nrem[:, 0], sleep_fold_nrem[:, 1], 'b.')
    ax3.plot(sleep_fold_rem[:, 0], sleep_fold_rem[:, 1], 'y.')
    plt.show()

    plt.plot(awake_fold[:, 0], awake_fold[:, 1], 'r.')
    plt.plot(sleep_fold_nrem[:, 0], sleep_fold_nrem[:, 1], 'b.')
    plt.plot(sleep_fold_rem[:, 0], sleep_fold_rem[:, 1], 'y.')
    plt.show()
    
def plot_signal(data):
    time = np.arange(len(data)) / SAMPLE_PER_SEC
    fig, ax = plt.subplots(1, 1, figsize=(12, 4))
    plt.plot(time, data, lw=1.5, color='k')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Voltage')
    plt.xlim([time.min(), time.max()])
    plt.title('N3 sleep EEG data (F3)')
    
def get_data_of_rat(X, y, i):
    sample_cnt = int(X.shape[0] / 3)
    if i == 0:
        return X[:sample_cnt, :], y[:sample_cnt]
    if i == 1:
        return X[sample_cnt: 2 * sample_cnt, :], y[sample_cnt: 2 * sample_cnt]
    if i == 2:
        return X[2*sample_cnt:, :], y[2*sample_cnt:]
    

In [6]:
# Load the data
# train
train_emg = os.path.join(dir_path,"data/train/train_emg.csv")
train_eeg_1 = os.path.join(dir_path,"data/train/train_eeg1.csv")
train_eeg_2 = os.path.join(dir_path,"data/train/train_eeg2.csv")

# trest
test_emg = os.path.join(dir_path,"data/test/test_emg.csv")
test_eeg_1 = os.path.join(dir_path,"data/test/test_eeg1.csv")
test_eeg_2 = os.path.join(dir_path,"data/test/test_eeg2.csv")

# labels
train_target = os.path.join(dir_path,'data/train/train_labels.csv')

x_train_emg = get_train_from_csv(train_emg) #List of numpy arrays
x_train_eeg_1 = get_train_from_csv(train_eeg_1) #List of numpy arrays
x_train_eeg_2 = get_train_from_csv(train_eeg_2) #List of numpy arrays
y_train = get_target_from_csv(train_target) #Numpy array of labels

x_test_emg = get_train_from_csv(test_emg) #List of numpy arrays
x_test_eeg_1 = get_train_from_csv(test_eeg_1) #List of numpy arrays
x_test_eeg_2 = get_train_from_csv(test_eeg_2) #List of numpy arrays

print(f"Class 1: {np.sum(y_train == 1)}")
print(f"Class 2: {np.sum(y_train == 2)}")
print(f"Class 3: {np.sum(y_train == 3)}")
# compute features
x_train_emg_feat = get_features_emg(x_train_emg)
x_train_eeg_1_feat = get_features_eeg(x_train_eeg_1) 
#x_train_emg_feat_spec = get_features_emg_wavelet(x_train_emg)
x_train_eeg_1_feat_spec = get_features_emg_wavelet(x_train_eeg_1)
x_test_emg_feat = get_features_emg(x_test_emg)
x_test_eeg_1_feat = get_features_eeg(x_test_eeg_1)

Reading /home/ax/master/2018_02/ml/task5/data/train/train_emg.csv
Reading /home/ax/master/2018_02/ml/task5/data/train/train_eeg1.csv
Reading /home/ax/master/2018_02/ml/task5/data/train/train_eeg2.csv
Reading /home/ax/master/2018_02/ml/task5/data/test/test_emg.csv
Reading /home/ax/master/2018_02/ml/task5/data/test/test_eeg1.csv
Reading /home/ax/master/2018_02/ml/task5/data/test/test_eeg2.csv
Class 1: 34114
Class 2: 27133
Class 3: 3553


In [29]:
x_test_emg_feat = get_features_emg(x_test_emg)
x_test_eeg_1_feat = get_features_eeg(x_test_eeg_1)

In [20]:
# CROSS VALDATION
X = x_train_eeg_1_
scaler = StandardScaler().fit(X)

kfold = KFold(n_splits=3, shuffle=False, random_state=seed)

# classifiers


print("Start")
bmac_scores = []

for train, valid in kfold.split(X):
    # get the folds
    X_train_fold_emg = X[train]
    y_train_fold = y_train[train]
    
    X_valid_fold_emg = X[valid]
    y_valid_fold = y_train[valid]

    # get the training for the awake/sleep clf
    rem_fold = X_train_fold_emg[y_train_fold == 2]
    nrem_fold = X_train_fold_emg[y_train_fold == 3]
    X_train_fold_rem_nrem = np.concatenate((rem_fold, nrem_fold))
    y_train_fold_rem_nrem = np.concatenate((np.ones(rem_fold.shape[0], dtype=np.int32) * 2, np.ones(nrem_fold.shape[0], dtype=np.int32) * 3))
    
    rem_fold = X_valid_fold_emg[y_valid_fold == 2]
    nrem_fold = X_valid_fold_emg[y_valid_fold == 3]
    X_valid_fold_rem_nrem = np.concatenate((rem_fold, nrem_fold))
    y_valid_fold_rem_nrem = np.concatenate((np.ones(rem_fold.shape[0], dtype=np.int32) * 2, np.ones(nrem_fold.shape[0], dtype=np.int32) * 3))
    
    # scale 
    #X_train_fold_awake_sleep = scaler.transform(X_train_fold_awake_sleep)
    #X_valid_fold_awake_sleep = scaler.transform(X_valid_fold_awake_sleep)
    
    # class weights
    classes = np.array([2,3])
    weights = compute_class_weight("balanced", classes, y_train_fold_rem_nrem)
    cw = dict(zip(classes, weights))

# classifiers

    rf_eeg = RandomForestClassifier(n_estimators=100, random_state=seed, n_jobs=-1, class_weight=cw, verbose=False)
    
    
    # fit classifier
    rf_eeg.fit(X_train_fold_rem_nrem, y_train_fold_rem_nrem)
    
    y_pred = rf_eeg.predict(X_valid_fold_rem_nrem)
    
    bmac_score = balanced_accuracy_score(y_valid_fold_rem_nrem, y_pred)
    print(f"{len(bmac_scores)}: current balanced_accuracy_score: {bmac_score}")

    bmac_scores.append(bmac_score)

print("========================================")
print(f"RFC BMAC avg score {np.mean(bmac_scores)} +/- {np.std(bmac_scores)}" )


Start
0: current balanced_accuracy_score: 0.8504817418247281
1: current balanced_accuracy_score: 0.8122146011098764
2: current balanced_accuracy_score: 0.8556966707903388
RFC BMAC avg score 0.8394643379083145 +/- 0.019385733374781958


In [24]:
# CROSS VALDATION
scaler = StandardScaler().fit(x_train_emg_feat)

kfold = KFold(n_splits=3, shuffle=False, random_state=seed)

# classifiers
rf_emg = RandomForestClassifier(n_estimators=5, random_state=seed, n_jobs=-1, verbose=False)

print("Start")
bmac_scores = []

for train, valid in kfold.split(x_train_emg_feat):
    X_train_fold_emg = x_train_emg_feat[train]
    X_train_fold_eeg = x_train_eeg_1_feat[train]
    y_train_fold = y_train[train]
    
    X_valid_fold_emg = x_train_emg_feat[valid]
    X_valid_fold_eeg = x_train_eeg_1_feat[valid]
    y_valid_fold = y_train[valid]
    
    # train scalers
    scaler = StandardScaler().fit(x_train_emg_feat)
    scaler_eeg = StandardScaler().fit(x_train_eeg_1_feat)


    # get the training for the awake/sleep clf
    awake_fold = X_train_fold_emg[y_train_fold == 1]
    sleep_fold = X_train_fold_emg[y_train_fold != 1]
    X_train_fold_awake_sleep = np.concatenate((awake_fold, sleep_fold))
    y_train_fold_awake_sleep = np.concatenate((np.ones(awake_fold.shape[0], dtype=np.int32), np.zeros(sleep_fold.shape[0], dtype=np.int32)))

    # get the training for nrem and rem sleep
    nrem_fold = X_train_fold_eeg[y_train_fold == 2]
    rem_fold = X_train_fold_eeg[y_train_fold == 3]
    X_train_fold_sleep_type = np.concatenate((nrem_fold, rem_fold))
    y_train_sleep_type = np.concatenate((np.ones(nrem_fold.shape[0], dtype=np.int32) * 2, np.ones(rem_fold.shape[0], dtype=np.int32) * 3))

    # scale 
    X_train_fold_awake_sleep = scaler.transform(X_train_fold_awake_sleep)
    X_train_fold_sleep_type = scaler_eeg.transform(X_train_fold_sleep_type)

    # class weights
    classes = np.array([2,3])
    weights = compute_class_weight("balanced", classes, y_train_sleep_type)
    cw = dict(zip(classes, weights))

    # classifiers
    rf_emg = RandomForestClassifier(n_estimators=100, random_state=seed, n_jobs=-1, verbose=False)
    rf_eeg = RandomForestClassifier(n_estimators=100, random_state=seed, n_jobs=-1, class_weight=cw, verbose=False)

    # fit classifier
    rf_emg.fit(X_train_fold_awake_sleep, y_train_fold_awake_sleep)
    rf_eeg.fit(X_train_fold_sleep_type, y_train_sleep_type)

    # scale validation
    X_valid_fold_emg = scaler.transform(X_valid_fold_emg)
    X_valid_fold_eeg = scaler_eeg.transform(X_valid_fold_eeg)

    y_pred = rf_emg.predict(X_valid_fold_emg)

    # compute where y_pred is zero 0 => sleep
    X_fold_valid_emg_sleep = X_valid_fold_eeg[y_pred == 0]
    sleep_index = np.argwhere(y_pred == 0)

    # predicted the type of sleep
    y_sleep = rf_eeg.predict(X_fold_valid_emg_sleep)

    # compute the final prediction vector
    j = 0
    for i in sleep_index:
        y_pred[i] = y_sleep[j]
        j +=1

    
    # get the folds
    X_train_fold_emg = x_train_emg_feat[train]
    y_train_fold = y_train[train]
    
    X_valid_fold_emg = x_train_emg_feat[valid]
    y_valid_fold = y_train[valid]

    # get the training for the awake/sleep clf
    awake_fold = X_train_fold_emg[y_train_fold == 1]
    sleep_fold = X_train_fold_emg[y_train_fold != 1]
    X_train_fold_awake_sleep = np.concatenate((awake_fold, sleep_fold))
    y_train_fold_awake_sleep = np.concatenate((np.ones(awake_fold.shape[0], dtype=np.int32), np.zeros(sleep_fold.shape[0], dtype=np.int32)))
    
    awake_fold = X_valid_fold_emg[y_valid_fold == 1]
    sleep_fold = X_valid_fold_emg[y_valid_fold != 1]
    X_valid_fold_awake_sleep = np.concatenate((awake_fold, sleep_fold))
    y_valid_fold_awake_sleep = np.concatenate((np.ones(awake_fold.shape[0], dtype=np.int32), np.zeros(sleep_fold.shape[0], dtype=np.int32)))
    
    # scale 
    #X_train_fold_awake_sleep = scaler.transform(X_train_fold_awake_sleep)
    #X_valid_fold_awake_sleep = scaler.transform(X_valid_fold_awake_sleep)
    
    # fit classifier
    rf_emg.fit(X_train_fold_awake_sleep, y_train_fold_awake_sleep)
    
    y_pred = rf_emg.predict(X_valid_fold_awake_sleep)
    
    bmac_score = balanced_accuracy_score(y_valid_fold_awake_sleep, y_pred)
    print(f"{len(bmac_scores)}: current balanced_accuracy_score: {bmac_score}")

    bmac_scores.append(bmac_score)

print("========================================")
print(f"RFC BMAC avg score {np.mean(bmac_scores)} +/- {np.std(bmac_scores)}" )



/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Start


/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


0: current balanced_accuracy_score: 0.9015153753143077


/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


1: current balanced_accuracy_score: 0.9636107703645397


/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


2: current balanced_accuracy_score: 0.9112481304578569
RFC BMAC avg score 0.9254580920455681 +/- 0.027269051784691307


In [30]:
# FINAL PREDICTION
scaler = StandardScaler().fit(x_train_emg_feat)
scaler_eeg = StandardScaler().fit(x_train_eeg_1_feat)

# class weights
nrem_fold = x_train_emg_feat[y_train == 2]
rem_fold = x_train_emg_feat[y_train == 3]
X_train_fold_sleep_type = np.concatenate((nrem_fold, rem_fold))
y_train_sleep_type = np.concatenate((np.ones(nrem_fold.shape[0], dtype=np.int32) * 2, np.ones(rem_fold.shape[0], dtype=np.int32) * 3))
classes = np.array([2,3])
weights = compute_class_weight("balanced", classes, y_train_sleep_type)
cw = dict(zip(classes, weights))

# classifiers
rf_emg = RandomForestClassifier(n_estimators=100, random_state=seed, n_jobs=-1, verbose=False)
rf_eeg = RandomForestClassifier(n_estimators=100, random_state=seed, n_jobs=-1, class_weight=cw, verbose=False)

# get the training for the awake/sleep clf
awake = x_train_emg_feat[y_train == 1]
sleep = x_train_emg_feat[y_train != 1]
X_train_awake_sleep = np.concatenate((awake, sleep))
y_train_awake_sleep = np.concatenate((np.ones(awake.shape[0], dtype=np.int32), np.zeros(sleep.shape[0], dtype=np.int32)))


# get the training for nrem and rem sleep
nrem = x_train_eeg_1_feat[y_train == 2]
rem = x_train_eeg_1_feat[y_train == 3]
X_train_sleep_type = np.concatenate((nrem, rem))
y_train_sleep_type = np.concatenate((np.ones(nrem.shape[0], dtype=np.int32) * 2, np.ones(rem.shape[0], dtype=np.int32) * 3))

# scale 
X_train_awake_sleep = scaler.transform(X_train_awake_sleep)
X_train_sleep_type = scaler_eeg.transform(X_train_sleep_type)
X_test_emg = scaler.transform(x_test_emg_feat)
X_test_eeg_1 = scaler_eeg.transform(x_test_eeg_1_feat)

# fit classifier
rf_emg.fit(X_train_awake_sleep, y_train_awake_sleep)
rf_eeg.fit(X_train_sleep_type, y_train_sleep_type)

# prediction
y_pred = rf_emg.predict(X_test_emg)

# compute where y_pred is zero 0 => sleep
X_test_eeg_1_sleep = X_test_eeg_1[y_pred == 0]
sleep_index = np.argwhere(y_pred == 0)

# predicted the type of sleep
y_sleep_type = rf_eeg.predict(X_test_eeg_1_sleep)

# compute the final prediction vector
j = 0
for i in sleep_index:
    y_pred[i] = y_sleep_type[j]
    j +=1

/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ax/miniconda3/envs/aml/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [31]:
submission_name = "ax_multi_stage_v2.csv"

print(f"Class 1: {np.sum(y_pred == 1)}")
print(f"Class 2: {np.sum(y_pred == 2)}")
print(f"Class 3: {np.sum(y_pred == 3)}")

y_pred_df = pd.DataFrame(y_pred)
y_pred_df = y_pred_df.assign(Id=list(range(y_pred.shape[0])))
y_pred_df.columns = ['y', 'Id']
display(y_pred_df)


submission_folder = os.path.join(dir_path,"submissions/")
csv_file = submission_folder + submission_name

with open(csv_file, 'w') as csv:
    y_pred_df.to_csv(csv,index = False)
"""
Class 1: 23933
Class 2: 18553
Class 3: 714
"""

Class 1: 21153
Class 2: 21157
Class 3: 890


,y,Id
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4
5,1,5
6,1,6
7,1,7
8,1,8
9,1,9


'\nClass 1: 23933\nClass 2: 18553\nClass 3: 714\n'